In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.exceptions import UndefinedMetricWarning
import warnings
import time
from sklearn.svm import SVC
import random
warnings.filterwarnings("ignore")
warnings.resetwarnings()
warnings.filterwarnings("ignore", category=FutureWarning)


In [7]:
df=pd.read_csv("/Users/saikarthik/Desktop/UROP/data.csv")

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [8]:
df.dropna(inplace=True)


In [9]:
df.drop_duplicates(inplace=True)


In [22]:
c0=0
c1=0
for i in  df["Label"]:
    if i== 0 :
        c0=c0+1
    elif i==1:
        c1=c1+1
print(c0,c1)            
        

14272 2046


In [11]:
for column in df.columns:
    if df[column].dtype == 'object':  
        df[column] = df[column].replace(to_replace='.*', value=0, regex=True)

In [10]:
cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(columns=cols_to_drop)


In [33]:
df.columns

Index(['SrcBytes', 'DstBytes', 'SrcLoad', 'DstLoad', 'SIntPkt', 'DIntPkt',
       'SIntPktAct', 'SrcJitter', 'DstJitter', 'sMaxPktSz', 'dMaxPktSz',
       'sMinPktSz', 'Dur', 'TotPkts', 'TotBytes', 'Load', 'Loss', 'pLoss',
       'pSrcLoss', 'pDstLoss', 'Rate', 'Packet_num', 'Temp', 'SpO2',
       'Pulse_Rate', 'SYS', 'DIA', 'Heart_rate', 'Resp_Rate', 'ST', 'Label'],
      dtype='object')

# Random Forest 

### Before GA and HC

In [13]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

start_time = time.time()

clf.fit(X_train, y_train)

end_time = time.time()

y_pred = clf.predict(X_test)

execution_time = end_time - start_time

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Execution Time: {execution_time:.2f} seconds")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Execution Time: 2.00 seconds
Accuracy: 0.9347426470588235
Precision: 0.981042654028436
Recall: 0.49759615384615385
F1-Score: 0.6602870813397129
Confusion Matrix:
 [[2844    4]
 [ 209  207]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96      2848
           1       0.98      0.50      0.66       416

    accuracy                           0.93      3264
   macro avg       0.96      0.75      0.81      3264
weighted avg       0.94      0.93      0.93      3264



### After applying Genetic Algorithm

In [ ]:


X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]



for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()  
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = RandomForestClassifier(n_estimators=100, random_state=42)
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")


print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)




print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)



accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)


print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)


### After applying Hill climb on Random forest

In [23]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()  
X_test = X_test.to_numpy()  

def objective_function(features):
    start_time = time.time()
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])
    end_time = time.time()

    return accuracy_score(y_test, y_pred)

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:  
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score

num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)
elapsed_time = end_time - start_time

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")

Selected Features: [0, 1, 2, 3, 10, 11, 12, 13, 14, 15, 18, 20, 21, 24, 25, 26, 27, 28, 29, 31, 32]
Best Score: 0.9589460784313726
Accuracy: 0.9589460784313726
Precision: 0.9595113439439142
Recall: 0.9589460784313726
F1 Score: 0.9559702794086331
Confusion Matrix:
 [[2840    8]
 [ 126  290]]
Time taken: 2.0015292167663574 seconds


# --------------------------------------------------
# SVM

### Before GA and HC

In [24]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', random_state=42)

start_time = time.time()

svm.fit(X_train, y_train)

end_time = time.time()

y_pred_svm = svm.predict(X_test)

execution_time = end_time - start_time

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_score_svm = f1_score(y_test, y_pred_svm)
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

print(f"Execution Time (SVM): {execution_time:.2f} seconds")
print("Accuracy (SVM):", accuracy_svm)
print("Precision (SVM):", precision_svm)
print("Recall (SVM):", recall_svm)
print("F1-Score (SVM):", f1_score_svm)
print("Confusion Matrix (SVM):\n", conf_matrix_svm)
print("Classification Report (SVM):\n", classification_rep_svm)


Execution Time (SVM): 364.24 seconds
Accuracy (SVM): 0.9283088235294118
Precision (SVM): 0.989247311827957
Recall (SVM): 0.4423076923076923
F1-Score (SVM): 0.6112956810631229
Confusion Matrix (SVM):
 [[2846    2]
 [ 232  184]]
Classification Report (SVM):
               precision    recall  f1-score   support

           0       0.92      1.00      0.96      2848
           1       0.99      0.44      0.61       416

    accuracy                           0.93      3264
   macro avg       0.96      0.72      0.79      3264
weighted avg       0.93      0.93      0.92      3264



In [27]:
X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 50
generations = 100
mutation_rate = 0.01

def evaluate_features(features):
    clf = SVC(kernel='linear', random_state=42)
    clf.fit(X_train.iloc[:, features], y_train)
    y_pred = clf.predict(X_test.iloc[:, features])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

population = [np.random.choice([0, 1], size=X_train.shape[1]) for _ in range(population_size)]

for generation in range(generations):
    fitness_scores = [evaluate_features(individual) for individual in population]
    num_parents = int(population_size * 0.2)
    parents = np.argsort(fitness_scores)[-num_parents]
    parents = parents.tolist()
    new_population = []
    for _ in range(population_size - num_parents):
        parent1 = population[np.random.choice(parents)]
        parent2 = population[np.random.choice(parents)]
        crossover_point = np.random.randint(0, X_train.shape[1])
        child = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
        mutation = np.random.rand(X_train.shape[1]) < mutation_rate
        child = (child + mutation) % 2
        new_population.append(child)
    population = np.vstack((population[parents], new_population))

best_individual = population[np.argmax(fitness_scores)]
selected_features = np.where(best_individual == 1)[0]

start_time_with_ga = time.time()
clf_with_ga = SVC(kernel='linear', random_state=42)
clf_with_ga.fit(X_train.iloc[:, selected_features], y_train)
y_pred_with_ga = clf_with_ga.predict(X_test.iloc[:, selected_features])
end_time_with_ga = time.time()

time_taken_with_ga = end_time_with_ga - start_time_with_ga

confusion_matrix_with_ga = confusion_matrix(y_test, y_pred_with_ga)

print("Time taken for Model with Genetic Algorithm Feature Selection:", time_taken_with_ga, "seconds")

print("\nConfusion Matrix for Model with Genetic Algorithm Feature Selection:")
print(confusion_matrix_with_ga)

accuracy_with_ga = accuracy_score(y_test, y_pred_with_ga)
precision_with_ga = precision_score(y_test, y_pred_with_ga)
recall_with_ga = recall_score(y_test, y_pred_with_ga)
f1_score_with_ga = f1_score(y_test, y_pred_with_ga)

print("\nModel with Genetic Algorithm Feature Selection Metrics:")
print("Accuracy (with GA):", accuracy_with_ga)
print("Precision (with GA):", precision_with_ga)
print("Recall (with GA):", recall_with_ga)
print("F1 Score (with GA):", f1_score_with_ga)

Time taken for Model with Genetic Algorithm Feature Selection: 478.169570684433 seconds

Confusion Matrix for Model with Genetic Algorithm Feature Selection:
[[2840    8]
 [ 234  182]]

Model with Genetic Algorithm Feature Selection Metrics:
Accuracy (with GA): 0.9258578431372549
Precision (with GA): 0.9578947368421052
Recall (with GA): 0.4375
F1 Score (with GA): 0.6006600660066007


### After Hill Climb 

In [ ]:
X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

start_time = time.time()

def objective_function(features):
    clf = SVC(kernel='linear', random_state=42)  # Use linear kernel for SVM
    clf.fit(X_train[:, features], y_train)
    y_pred = clf.predict(X_test[:, features])

    return accuracy_score(y_test, y_pred)

def hill_climbing_feature_selection(num_features, num_iterations):
    current_features = list(range(num_features))
    current_score = objective_function(current_features)

    for _ in range(num_iterations):
        neighbor = current_features.copy()
        if len(neighbor) > 1:
            feature_to_remove = random.choice(neighbor)
            neighbor.remove(feature_to_remove)
        else:
            features_not_in_subset = list(set(range(num_features)) - set(neighbor))
            if features_not_in_subset:
                feature_to_add = random.choice(features_not_in_subset)
                neighbor.append(feature_to_add)
        neighbor_score = objective_function(neighbor)
        if neighbor_score > current_score:
            current_features = neighbor
            current_score = neighbor_score

    return current_features, current_score

num_features = X_train.shape[1]
num_iterations = 100

selected_features, best_score = hill_climbing_feature_selection(num_features, num_iterations)

end_time = time.time()
elapsed_time = end_time - start_time

clf = SVC(kernel='linear', random_state=42) 
clf.fit(X_train[:, selected_features], y_train)
y_pred = clf.predict(X_test[:, selected_features])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("Selected Features:", selected_features)
print("Best Score:", best_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
print("Time taken:", elapsed_time, "seconds")